In [1]:
import tensorflow as tf
import numpy as np
from numpy import linalg as LA
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder(tf.float32, shape=[784, None])
y_true = tf.placeholder(tf.float32, shape=[10, None])
W = tf.Variable(tf.zeros([10,784]))
# b = tf.Variable(tf.zeros([10,1]))
y_pred = tf.matmul(W, x) 
y_pred.get_shape()

TensorShape([Dimension(10), Dimension(None)])

In [4]:
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))
# optimzr = tf.train.AdamOptimizer(1e-4).minimize(loss)

# correct_prediction = tf.equal(tf.argmax(y_pred, 0), tf.argmax(y_true, 0))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [5]:
def tf_frobenius_norm(M):
    return tf.reduce_sum(M ** 2) ** 0.5

# # to implement nuclear norm
# def tf_nuclear_norm(M):
#     st, ut, vt = tf.svd(M)
#     uk = ut[:, 0]
#     vk = vt[:, 0]
#     print(M.shape, ut.shape, vt.shape, tf.transpose(vk).shape)
#     sk = tf.matmul(uk, tf.transpose(vk))
#     return sk

def Sgdnm(grad, wt):
    return (grad / tf_frobenius_norm(grad))

def Cgd_Fn(grad, wt):
    return ((1 - alpha ) / alpha) * (wt + lam1 * grad / tf_frobenius_norm(grad))

def Cgd_Nn(grad, wt):
    return ((1 - alpha ) / alpha) * (wt - lam2 * tf_nuclear_norm(grad))


In [6]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

# hyper-parameters
#alpha = tf.placeholder_with_default(tf.constant(0.5), tf.constant(0.001).shape)
global_step = tf.Variable(0, trainable=False)
start_train = 0.999
# k=1, start_train = 1, decay_rate = 1 ---> 1/t learning rate
k = 1
alpha = tf.train.inverse_time_decay(start_train, global_step, k, 0)
lam1 = tf.placeholder_with_default(tf.constant(4.0), tf.constant(10000.0).shape)
lam2 = tf.placeholder_with_default(tf.constant(100.0), tf.constant(100.0).shape)

# Gradient Descent optimizer
opt = tf.train.GradientDescentOptimizer(learning_rate = alpha)

# Compute the gradients for a list of variables.
grads_and_vars = opt.compute_gradients(loss)

# SGD update
# gv_sgd = [(gv[0], gv[1]) for gv in grads_and_vars]
# optimizer_gv_sgd = opt.apply_gradients(gv_sgd, global_step=global_step)
# qa = grads_and_vars[0][0]
# qb = tf_frobenius_norm(grads_and_vars[0][0])
# qc = grads_and_vars[0][0] / tf_frobenius_norm(grads_and_vars[0][0])
# qd = Cgd_Fn(grads_and_vars[0][0], grads_and_vars[0][1])
# qe = grads_and_vars[0][1]
# qf = ((1 - alpha ) / alpha)*(grads_and_vars[0][1] - lam1 * grads_and_vars[0][0] / tf_frobenius_norm(grads_and_vars[0][0]))
# qg = qe - alpha * qa

# # Normalized SGD update
# gv_nsgd = [(Sgdnm(gv[0], gv[1]), gv[1]) for gv in grads_and_vars]
# optimizer_gv_nsgd = opt.apply_gradients(gv_nsgd, global_step=global_step)
# qa = grads_and_vars[0][0]
# qb = tf_frobenius_norm(grads_and_vars[0][0])
# qc = grads_and_vars[0][0] / tf_frobenius_norm(grads_and_vars[0][0])
# qd = Cgd_Fn(grads_and_vars[0][0], grads_and_vars[0][1])
# qe = grads_and_vars[0][1]
# qf = Sgdnm(grads_and_vars[0][0], grads_and_vars[0][1])
# qg = qe - alpha * qf


# CGD with FN
gv_cgd_fn = [(Cgd_Fn(gv[0], gv[1]), gv[1]) for gv in grads_and_vars]
optimizer_gv_cgd_fn = opt.apply_gradients(gv_cgd_fn, global_step=global_step)
qa = grads_and_vars[0][0]
qb = tf_frobenius_norm(grads_and_vars[0][0])
qc = grads_and_vars[0][0] / tf_frobenius_norm(grads_and_vars[0][0])
qd = Cgd_Fn(grads_and_vars[0][0], grads_and_vars[0][1])
qe = grads_and_vars[0][1]
qf = ((1 - alpha ) / alpha)*(grads_and_vars[0][1] - lam1 * grads_and_vars[0][0] / tf_frobenius_norm(grads_and_vars[0][0]))
qg = qe - alpha * qf

# # CGD with NN
# gv_cgd_nn = [(Cgd_Nn(gv[0], gv[1]), gv[1]) for gv in grads_and_vars]
# optimizer_gv_cgd_Nn = opt.apply_gradients(gv_cgd_nn)

correct_prediction = tf.equal(tf.argmax(y_pred, 0), tf.argmax(y_true, 0))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Nuclear Norm

In [7]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for i in range(1000):
#         batch = mnist.train.next_batch(100)
#         feed_dict = {x: np.transpose(batch[0]), y_true: np.transpose(batch[1])}
#         train_accuracy, loss_, gv_, W_ = sess.run([accuracy, loss, gv_cgd_nn, W], feed_dict)
# #         qa_,qb_,qc_,qd_,qe_,qf_, qg_ = sess.run([qa,qb,qc,qd,qe,qf,qg], feed_dict)
            
#         if i % 100 == 0:
#             print('train_accuracy', train_accuracy, loss_)
# #             print('frob_nrom: qg =', LA.norm(qg_), 'qe =', LA.norm(qe_), 'qf =', LA.norm(qf_))
# #             print('gradient', LA.norm(qa_))
#         sess.run(optimizer_gv_cgd_nn, feed_dict)
    
#     feed_dict={x: np.transpose(mnist.test.images), y_true: np.transpose(mnist.test.labels)}
#     test_accuracy = sess.run(accuracy, feed_dict)
#     print('test_accuracy', test_accuracy)

# Frobenius norm

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10000):
        batch = mnist.train.next_batch(100)
        feed_dict = {x: np.transpose(batch[0]), y_true: np.transpose(batch[1])}
        train_accuracy, loss_, gv_, W_ = sess.run([accuracy, loss, gv_cgd_fn, W], feed_dict)
        qa_,qb_,qc_,qd_,qe_,qf_, qg_ = sess.run([qa,qb,qc,qd,qe,qf,qg], feed_dict)
        alpha_ = sess.run([alpha], feed_dict)
            
        if i % 100 == 0:
            print('train_accuracy=', train_accuracy, 'loss value =',loss_)
            print('frob_nrom of iterates: w(t+1) =', LA.norm(qg_), 'w(t) =', LA.norm(qe_), 's(t) =', LA.norm(qf_))
            print('alpha', alpha_)
#             print('gradient', LA.norm(qa_))
        sess.run(optimizer_gv_cgd_fn, feed_dict)
    
    feed_dict={x: np.transpose(mnist.test.images), y_true: np.transpose(mnist.test.labels)}
    test_accuracy = sess.run(accuracy, feed_dict)
    print('test_accuracy', test_accuracy)

train_accuracy= 0.1 loss value = 46.0517
frob_nrom of iterates: w(t+1) = 0.00399995 w(t) = 0.0 s(t) = 0.00400395
alpha [0.99900001]
train_accuracy= 0.71 loss value = 44.3283
frob_nrom of iterates: w(t+1) = 0.342185 w(t) = 0.346091 s(t) = 0.00431553
alpha [0.99900001]
train_accuracy= 0.7 loss value = 42.8548
frob_nrom of iterates: w(t+1) = 0.655422 w(t) = 0.659551 s(t) = 0.00458885
alpha [0.99900001]
train_accuracy= 0.63 loss value = 42.2044
frob_nrom of iterates: w(t+1) = 0.937463 w(t) = 0.942003 s(t) = 0.00486987
alpha [0.99900001]
train_accuracy= 0.65 loss value = 41.9642
frob_nrom of iterates: w(t+1) = 1.19232 w(t) = 1.19713 s(t) = 0.00511397
alpha [0.99900001]
train_accuracy= 0.65 loss value = 41.5802
frob_nrom of iterates: w(t+1) = 1.4237 w(t) = 1.42881 s(t) = 0.0053486
alpha [0.99900001]
train_accuracy= 0.71 loss value = 41.9958
frob_nrom of iterates: w(t+1) = 1.63274 w(t) = 1.63787 s(t) = 0.00549534
alpha [0.99900001]
train_accuracy= 0.71 loss value = 41.8092
frob_nrom of iterat